<a href="https://colab.research.google.com/github/mikeroguez/experimento/blob/main/Tareas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Codigo para normalizar**

In [8]:
import pandas as pd
import numpy as np
import html
from typing import Tuple, Optional

def get_semester_bounds(commitment_date: pd.Timestamp) -> Tuple[pd.Timestamp, pd.Timestamp]:
    """Determina los límites del semestre basado en la fecha de compromiso."""
    if pd.isnull(commitment_date):
        return pd.NaT, pd.NaT

    year, month = commitment_date.year, commitment_date.month

    if 2 <= month <= 7:
        return pd.Timestamp(year, 2, 1), pd.Timestamp(year, 7, 31)
    elif month == 1:
        return pd.Timestamp(year - 1, 8, 1), pd.Timestamp(year, 1, 31)
    else:
        return pd.Timestamp(year, 8, 1), pd.Timestamp(year + 1, 1, 31)

def process_data(input_path: str, output_path: str) -> pd.DataFrame:
    """Procesa el dataset de tareas académicas y guarda el resultado."""
    FINAL_COLUMNS = [
        'curso', 'groupKey', 'email', 'id_actividad', 'actividad',
        'fecha_compromiso', 'fecha_de_entrega', 'calificacion',
        'calificacion_ajustada', 'is_atypical',
        'calificacion_normalizada', 'calificacion_estandarizada',
        'dias_anticipacion', 'fuera_de_rango', 'fue_entregada', 'fue_evaluada', 'tipo_entrega'
    ]

    # Cargar y preparar los datos
    df = pd.read_csv(input_path)
    df.rename(columns={
        'Fecha_compromiso': 'fecha_compromiso',
        'Fecha_de_entrega': 'fecha_de_entrega',
        'Calificacion': 'calificacion',
        'Actividad': 'actividad'
    }, inplace=True)

    # Parsear fechas
    df['fecha_compromiso'] = pd.to_datetime(df['fecha_compromiso'], errors='coerce')
    df['fecha_de_entrega'] = pd.to_datetime(df['fecha_de_entrega'], errors='coerce')

    # Limpiar y normalizar la columna 'actividad'
    df['actividad'] = (
        df['actividad']
        .fillna("actividad desconocida")
        .apply(lambda x: html.unescape(str(x)))
        .str.replace(r'[^\x00-\x7F]+', '', regex=True)
    )

    # **Ajuste de valores atípicos con verificación**
    actividad_stats = df.groupby('id_actividad')['calificacion'].agg(
        p99=lambda x: np.percentile(x.dropna(), 99) if x.dropna().size > 0 else np.nan
    ).reset_index()

    df = df.merge(actividad_stats, on='id_actividad', how='left')

    # Crear columna `is_atypical` y ajustar valores atípicos
    df['is_atypical'] = (df['calificacion'] > df['p99']).astype(int)
    df['calificacion_ajustada'] = np.where(
        df['calificacion'] > df['p99'], df['p99'], df['calificacion']
    )

    # **Normalización basada en el máximo de la actividad**
    max_scores = df.groupby('id_actividad')['calificacion_ajustada'].max().reset_index()
    max_scores.rename(columns={'calificacion_ajustada': 'max_calificacion'}, inplace=True)
    df = df.merge(max_scores, on='id_actividad', how='left')

    df['calificacion_normalizada'] = np.where(
        df['calificacion_ajustada'].notna(),
        (df['calificacion_ajustada'] / df['max_calificacion'] * 10).round(2),
        np.nan  # Si no hay calificación, normalización también debe ser NaN
    )

    # **Estandarización (Z-score con corrección)**
    actividad_stats = df.groupby('id_actividad')['calificacion_ajustada'].agg(mean='mean', std='std', count='count')
    df = df.merge(actividad_stats, left_on='id_actividad', right_index=True, how='left')

    df['calificacion_estandarizada'] = np.where(
        df['calificacion_ajustada'].isna(),  # Si no hay calificación
        np.nan,  # Mantener NaN
        np.where(
            df['count'] == 1,  # Si solo hay una calificación en la actividad
            0,  # Z-score = 0 (único dato disponible)
            ((df['calificacion_ajustada'] - df['mean']) / np.where(df['std'] == 0, 1, df['std'])).round(2)  # Evitar división por 0
        )
    )

    # **Cálculo de días de anticipación**
    df['dias_anticipacion'] = (df['fecha_compromiso'] - df['fecha_de_entrega']).dt.days

    # **Cálculo de `fuera_de_rango`**
    df[['semester_start', 'semester_end']] = df['fecha_compromiso'].apply(lambda x: pd.Series(get_semester_bounds(x)))

    df['fuera_de_rango'] = df.apply(
        lambda x: (
            pd.notna(x['fecha_de_entrega']) and
            pd.notna(x['semester_start']) and
            pd.notna(x['semester_end']) and
            (x['fecha_de_entrega'] < x['semester_start'] or x['fecha_de_entrega'] > x['semester_end'])
        ),
        axis=1
    ).astype(int)  # Convertir a 0/1

    # **Indicadores de entrega y evaluación**
    df['fue_entregada'] = df['fecha_de_entrega'].notna().astype(int)
    df['fue_evaluada'] = df['calificacion'].notna().astype(int)

    # **Clasificación del tipo de entrega**
    df['tipo_entrega'] = np.select(
        [
            df['fecha_de_entrega'].isna(),
            df['dias_anticipacion'] > 7,
            df['dias_anticipacion'].between(0, 7),
            df['dias_anticipacion'] < 0
        ],
        ['faltante', 'temprana', 'puntual', 'tardía'],
        default='desconocida'
    )

    return df[FINAL_COLUMNS].copy()

if __name__ == "__main__":
    INPUT_PATH = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas.csv"
    OUTPUT_PATH = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_normalizado.csv"

    processed_df = process_data(INPUT_PATH, OUTPUT_PATH)
    processed_df.to_csv(OUTPUT_PATH, index=False)
    print(f"Dataset procesado guardado en: {OUTPUT_PATH}")


Dataset procesado guardado en: /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_normalizado.csv


**2. Código para consolidar**

In [9]:
import pandas as pd

# Cargar el dataset desde Google Drive
file_path = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_normalizado.csv"
data = pd.read_csv(file_path)

# Función para consolidar los datos con cálculos consistentes
def consolidate_data_fixed(data):
    # Agrupar y calcular métricas principales
    consolidated = data.groupby(['curso', 'email', 'groupKey']).agg(
        total_assignments=('id_actividad', 'count'),
        avg_assignment_normalized_score=('calificacion_normalizada', 'mean'),
        avg_assignment_estandard_score=('calificacion_estandarizada', 'mean'),
        max_assignment_normalized_score=('calificacion_normalizada', 'max'),
        min_assignment_normalized_score=('calificacion_normalizada', 'min'),
        max_assignment_estandard_score=('calificacion_estandarizada', 'max'),
        min_assignment_estandard_score=('calificacion_estandarizada', 'min'),
        score_variability=('calificacion_normalizada', 'std'),
        total_graded_assignments=('fue_evaluada', 'sum'),
        total_submitted_assignments=('fue_entregada', 'sum'),
        total_missing_assignments=('fue_entregada', lambda x: (x == 0).sum()),
        total_ungraded_assignments=('fue_evaluada', lambda x: (x == 0).sum()),
        total_out_of_range_assignments=('fuera_de_rango', 'sum'),
        total_late_assignments=('tipo_entrega', lambda x: (x == 'tardía').sum()),
        avg_submission_delay_days=(
            'dias_anticipacion',
            lambda x: x[data.loc[x.index, 'fue_entregada'] == 1].mean() if not x.empty else 0
        )
    ).reset_index()

    # Calcular tasas y métricas derivadas
    consolidated['assignment_submission_rate'] = (
        consolidated['total_submitted_assignments'] / consolidated['total_assignments']
    )
    consolidated['assignment_procrastination_rate'] = (
        consolidated['total_late_assignments'] / consolidated['total_submitted_assignments']
    )

    # Calcular rendimiento general ponderado
    w1, w2 = 0.5, 0.5
    consolidated['overall_assignment_performance'] = (
        (consolidated['avg_assignment_normalized_score'] * w1) +
        (consolidated['assignment_submission_rate'] * 10 * w2)
    )

    # Clasificar rendimiento general
    def classify_general_performance(overall_score):
        if overall_score >= 9.0:
            return 'Excelente'
        elif overall_score >= 8.0:
            return 'Bien'
        elif overall_score >= 6.0:
            return 'Suficiente'
        else:
            return 'Insuficiente'

    consolidated['overall_assignment_performance_classification'] = (
        consolidated['overall_assignment_performance'].apply(classify_general_performance)
    )

    # Revisar si todos los trabajos del grupo no fueron calificados
    group_status = consolidated.groupby(['curso', 'groupKey'])['total_graded_assignments'].sum().reset_index()
    group_status['delete'] = group_status['total_graded_assignments'].apply(lambda x: 1 if x == 0 else 0)

    # Unir el estado del grupo al DataFrame consolidado
    consolidated = pd.merge(
        consolidated, group_status[['curso', 'groupKey', 'delete']],
        on=['curso', 'groupKey'], how='left'
    )

    # Determinar si se debe marcar course_retake_probability como 1
    course_status = consolidated.groupby('curso').apply(
        lambda x: 1 if (x['total_submitted_assignments'] > 0).any() and
                        all(x.loc[x['total_submitted_assignments'] > 0, 'total_out_of_range_assignments'] > 0) and
                        abs(x['avg_submission_delay_days'].mean()) > 30
                  else 0
    ).reset_index(name='course_retake_probability')

    # Unir el estado del curso al DataFrame consolidado
    consolidated = pd.merge(consolidated, course_status, on='curso', how='left')

    return consolidated

# Aplicar la función para consolidar los datos
corrected_consolidated_data = consolidate_data_fixed(data)

# Guardar los datos consolidados corregidos
output_path_corrected = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_consolidado.csv"
corrected_consolidated_data.to_csv(output_path_corrected, index=False)

print(f"Archivo consolidado guardado en: {output_path_corrected}")


<ipython-input-9-c81e60e9ceae>:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  course_status = consolidated.groupby('curso').apply(


Archivo consolidado guardado en: /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_consolidado.csv


**3. Código para limpiar**

In [10]:
import pandas as pd

# Ruta del archivo de entrada y salida
input_file = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_consolidado.csv"
output_file = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_consolidado_cleaned.csv"

# Cargar el archivo consolidado
data = pd.read_csv(input_file)

# Filtrar registros donde delete = 1 y eliminar la columna 'delete'
def filter_data(data):
    # Filtrar registros donde delete != 1
    filtered_data = data[data['delete'] != 1].copy()

    # Eliminar la columna 'delete'
    filtered_data.drop(columns=['delete'], inplace=True)

    return filtered_data

# Manejar valores nulos e imputar
def handle_missing_values(data):
    # Imputar valores nulos en las columnas relevantes
    fill_zero_columns = [
        'avg_assignment_normalized_score', 'avg_assignment_estandard_score',
        'max_assignment_normalized_score', 'min_assignment_normalized_score',
        'max_assignment_estandard_score', 'min_assignment_estandard_score',
        'score_variability', 'avg_submission_delay_days',
        'assignment_procrastination_rate', 'overall_assignment_performance'
    ]
    data[fill_zero_columns] = data[fill_zero_columns].fillna(0)

    return data

# Aplicar el pipeline
data_filtered = filter_data(data)
data_cleaned = handle_missing_values(data_filtered)

# Guardar el archivo limpio
data_cleaned.to_csv(output_file, index=False)

print(f"Archivo limpio guardado en: {output_file}")

Archivo limpio guardado en: /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_consolidado_cleaned.csv


**4. Elimina cursos posiblemente reutilizados**

In [11]:
import pandas as pd

# Definir archivos de entrada y salida
input_file = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_consolidado_cleaned.csv"
output_file = "/content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_consolidado_cleaned_sin_cursos_reutilizados.csv"

# Cargar el dataset
data = pd.read_csv(input_file)

# Filtrar registros donde course_retake_probability != 1
data_filtered = data[data['course_retake_probability'] != 1]

# Eliminar la columna course_retake_probability
data_filtered = data_filtered.drop(columns=['course_retake_probability'])

# Guardar el resultado en un nuevo archivo
data_filtered.to_csv(output_file, index=False)

print(f"Archivo guardado en: {output_file}")

Archivo guardado en: /content/drive/MyDrive/Doctorado/Experimentos/Datos/DSv4.0/1. Para tratar/tareas/tareas_consolidado_cleaned_sin_cursos_reutilizados.csv
